In [ ]:
import pandas as pd
from multiprocessing import Pool
import zipfile
import requests
import io
import os
from functools import partial
import sys
import boto3
import pymongo
from ast import literal_eval
from pymongo import MongoClient
import pprint
import pdb

#sys.argv[1] = '010-mentions-translation.csv'
#sys.argv[2] = 'mentions-header.txt'
#sys.argv[3] = 'mention-json-cols.txt'

filename = '010-mentions-translation.csv'
filelist = list(filter(lambda x:x!='',open(filename,'r').read().split('\n')))
headers = open('mentions-header.txt','r').read().split()
collist = open('mentions-json-cols.txt','r').read().split()

def updateMongoMentions(headers, collist, fileurl):
    client = MongoClient('mongodb://172.31.47.117:27020')
    db = client['gdelt']
    collection = db['events']
    z = zipfile.ZipFile(io.BytesIO(requests.get(fileurl).content))
    df = pd.read_csv(z.open(fileurl.split('/')[-1][:-4]),delimiter='\t',names=headers,index_col=False).fillna('')[collist]
    df['MentionDocTranslationInfo'].replace('','srclc:eng')
    df['MentionDocTranslationInfo']=df['MentionDocTranslationInfo'].str[6:9]
    df = df.drop_duplicates()
    js = df.groupby('GlobalEventID',as_index='GlobalEventID').apply(lambda x:x.to_json(orient='records'))
    
    for i,j in enumerate(js):
        eventId = js.index[i]
        temp_mention = literal_eval(j[1:-1])
        pdb.stacktrace()
        if type(temp_mention)is tuple:
            collection.update_one({'GlobalEventID':eventId},{'$addToSet':{'mentions':{'$each':temp_mention}}})
        else:
            collection.update_one({'GlobalEventID':eventId},{'$addToSet':{'mentions':temp_mention}})

p = Pool(100)
res = p.map(partial(updateMongoMentions,headers,collist),filelist)

In [43]:
js.apply

'[{"GlobalEventID":719024871,"MentionDocTone":-2.9702970297,"MentionDocTranslationInfo":"","MentionIdentifier":"http:\\/\\/www.modbee.com\\/news\\/business\\/article192357844.html"},{"GlobalEventID":719024871,"MentionDocTone":-2.5316455696,"MentionDocTranslationInfo":"","MentionIdentifier":"http:\\/\\/www.fresnobee.com\\/news\\/business\\/article192357844.html"},{"GlobalEventID":719024871,"MentionDocTone":-2.5316455696,"MentionDocTranslationInfo":"","MentionIdentifier":"http:\\/\\/www.kansas.com\\/news\\/business\\/article192357844.html"}]'

In [3]:
import json
js = json.load(open('010.json','r'))

In [5]:
js[0]

{'GlobalEventID': 669477179,
 'MentionDocTone': 1.3806706114,
 'MentionDocTranslationInfo': 'spa',
 'MentionIdentifier': 'http:\\/\\/www.noticiasdenavarra.com\\/2018\\/06\\/30\\/vecinos\\/ribera-alta\\/lodosa-le-cede-el-antiguo-matadero-a-proteccion-civil'}

In [45]:
for doc in collection.find_one({'GlobalEventId':613269348}):
    pprint.pprint(doc)

TypeError: 'NoneType' object is not iterable

In [6]:
from pymongo import MongoClient
import pprint

In [2]:
client = MongoClient('mongodb://172.31.47.117:27020/ssl=true&ssl_keyfile=~/Downloads/mongodb_tram.pem1')
db = client['gdelt']
collection = db['events']

In [7]:
pprint.pprint(collection.find_one({'Day':'20180101'}))

KeyboardInterrupt: 